In [6]:
import numpy as np
import tensorflow as tf

## 变量作用域

#### variable_scope
v = tf.get_variable(name, shape, dtype, initializer)
tf.variable_scope(<scope_name>)

In [3]:
# tf.get_variable_scope().reuse == False
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
    v2 = tf.get_variable("v", [1])
assert v.name == "foo/v:0"

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-2-c59d364ceefb>", line 2, in <module>
    v = tf.get_variable("v", [1])
  File "/home/tfb/env/tensorflow2/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/tfb/env/tensorflow2/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2822, in run_ast_nodes
    if self.run_code(code, result):


In [4]:
with tf.variable_scope("foo", reuse=True) as scope:
    v = tf.get_variable("v", [1])
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
assert v1 == v

### 获取变量作用域

In [6]:
with tf.variable_scope("foo") as foo_scope:
    assert foo_scope.name == "foo"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name == "foo"

In [7]:
with tf.variable_scope("foo") as foo_scope:
    assert foo_scope.name == "foo"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name == "foo"

### 变量作用域的初始化

In [2]:
with tf.variable_scope("foo", initializer=tf.constant_initializer(0.4)):
    v = tf.get_variable("v", [1])
    assert v.eval() == 0.4
    w = tf.get_variable("w", [1], initializer=tf.constant_initializer(0.3))
    assert w.eval() == 0.3
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
        assert v.eval() == 0.4
    with tf.variable_scope("baz", initializer=tf.constant_initializer(0.2)):
        v = tf.get_variable("v", [1])
        assert v.eval() == 0.2

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [3]:
with tf.variable_scope("foo"):
    x = 1.0 + tf.get_variable("v", [1])
    assert x.op.name == "foo/add"

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-2-9a4c5229255a>", line 2, in <module>
    v = tf.get_variable("v", [1])
  File "/home/tfb/env/tensorflow2/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/tfb/env/tensorflow2/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2822, in run_ast_nodes
    if self.run_code(code, result):


### name_scope示例

In [4]:
with tf.variable_scope("foo"):
    with tf.name_scope("bar"):
        v = tf.get_variable("v", [1])
        b = tf.Variable(tf.zeros([1]), name='b')
        x = 1.0 + v
assert v.name == "foo/v:0"
assert b.name == "foo/bar/b:0"
assert x.op.name == "foo/bar/add"

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-2-9a4c5229255a>", line 2, in <module>
    v = tf.get_variable("v", [1])
  File "/home/tfb/env/tensorflow2/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/tfb/env/tensorflow2/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2822, in run_ast_nodes
    if self.run_code(code, result):


## 神经元函数

### 激活函数

In [2]:
tf.nn.relu()
tf.nn.sigmoid()
tf.nn.tanh()
tf.nn.elu()
tf.nn.bias_add()
tf.nn.crelu()
tf.nn.relu6()
tf.nn.softplus()
tf.nn.softsign()
tf.nn.dropout()

TypeError: relu() takes at least 1 argument (0 given)

In [3]:
a = tf.constant([[1.0, 2.0], [1.0, 2.0], [1.0, 2.0]])
sess = tf.Session()
print(sess.run(tf.sigmoid(a)))

[[ 0.7310586   0.88079703]
 [ 0.7310586   0.88079703]
 [ 0.7310586   0.88079703]]


In [4]:
-0= tf.constant([-1.0, 2.0])
with tf.Session() as sess:
    b = tf.nn.relu(a)

    print(sess.run(b))
    

[ 0.  2.]


In [5]:
a = tf.constant([[-1.0, 2.0, 3.0, 4.0]])
with tf.Session() as sess:
    b = tf.nn.dropout(a, 0.5, noise_shape = [1,4])
    print(sess.run(b))
    b = tf.nn.dropout(a, 0.5, noise_shape = [1,1])
    print(sess.run(b))

[[-0.  0.  6.  0.]]
[[-0.  0.  0.  0.]]


### 卷积函数

In [9]:
input_data = tf.Variable(np.random.rand(10, 9, 9, 3), dtype=np.float32)
filter_data = tf.Variable(np.random.rand(2, 2, 3, 2), dtype=np.float32)
y = tf.nn.conv2d(input_data, filter_data, strides=[1, 1, 1, 1], padding='SAME')
with tf.Session() as sess:
    print(sess.run(tf.shape(y)))

[10  9  9  2]


In [10]:
input_data = tf.Variable(np.random.rand(10, 9, 9, 3), dtype=np.float32)
filter_data = tf.Variable(np.random.rand(2, 2, 3, 5), dtype=np.float32)
y = tf.nn.depthwise_conv2d(input_data, filter_data, strides=[1, 1, 1, 1], padding='SAME')
with tf.Session() as sess:
    print(sess.run(tf.shape(y)))

[10  9  9 15]


In [11]:
input_data = tf.Variable(np.random.rand(10, 9, 9, 3), dtype=np.float32)
depthwise_filter = tf.Variable(np.random.rand(2, 2, 3, 5), dtype=np.float32)
pointwise_filter = tf.Variable(np.random.rand(1, 1, 15, 20), dtype=np.float32)
y = tf.nn.separable_conv2d(input_data, depthwise_filter, pointwise_filter, strides=[1, 1, 1, 1], padding='SAME')
with tf.Session() as sess:
    print(sess.run(tf.shape(y)))

[10  9  9 20]


In [13]:
input_data = tf.Variable(np.random.rand(1, 5, 5, 1), dtype=np.float32)
filter_data = tf.Variable(np.random.rand(3, 3, 1, 1), dtype=np.float32)
y = tf.nn.atrous_conv2d(input_data, filter_data, 2, padding='SAME')
with tf.Session() as sess:
    print(sess.run(tf.shape(y)))

[1 5 5 1]


In [15]:
x = tf.random_normal(shape=[1, 3, 3, 1])
kernel = tf.random_normal(shape=[2, 2, 3, 1])
y = tf.nn.conv2d_transpose(x, kernel, output_shape=[1, 5, 5, 3], strides=[1, 2, 2, 1], padding="SAME")
with tf.Session() as sess:
    print(sess.run(tf.shape(y)))

[1 5 5 3]


### 池化函数

In [16]:
input_data = tf.Variable(np.random.rand(10, 6, 6, 3), dtype=np.float32)
filter_data = tf.Variable(np.random.rand(2, 2, 3, 10), dtype=np.float32)
y = tf.nn.conv2d(input_data, filter_data, strides=[1, 1, 1, 1], padding='SAME')
output = tf.nn.avg_pool(value = y, ksize = [1, 2, 2, 1], strides = [1, 1, 1, 1], padding = 'SAME')
with tf.Session() as sess:
    print(sess.run(tf.shape(output)))

[10  6  6 10]


In [20]:
input_data = tf.Variable(np.random.rand(10, 6, 6, 3), dtype=np.float32)
filter_data = tf.Variable(np.random.rand(2, 2, 3, 10), dtype=np.float32)
y = tf.nn.conv2d(input_data, filter_data, strides=[1, 1, 1, 1], padding='SAME')
with tf.Session() as sess:
    print(sess.run(tf.shape(y)))

[10  6  6 10]


## 模型的存储与加载

### 加载数据及定义模型

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)